In [1]:
# 用窗口取数
import pandas as pd

# 参数设置
window_size = 28
step_size = 14

# 读取数据
input_file = '../DATA/labeled_tactile_dataset.csv'
output_file = '../DATA/labeled_window_dataset.csv'

df = pd.read_csv(input_file)

# 初始化存储窗口信息的列表
windowed_data = []

# 初始化 window_id
window_id = 0

# 循环以 step_size 步长遍历数据
for start in range(0, len(df) - window_size + 1, step_size):
    end = start + window_size
    window = df.iloc[start:end].copy()
    
    # 生成 window_id
    window['window_id'] = window_id
    window_id += 1  # 每次迭代增加 window_id
    
    # 判断窗口内 touch_type 是否全为 'NC'
    unique_touch_types = window['touch_type'].unique()
    if len(unique_touch_types) == 1 and unique_touch_types[0] == 'NC':
        window_touch_type = 'NC'
    else:
        # 如果不是全为 'NC'，取窗口内唯一的非 'NC' 的 touch_type
        non_nc_touch_types = [t for t in unique_touch_types if t != 'NC']
        window_touch_type = non_nc_touch_types[0] if non_nc_touch_types else 'NC'
    
    # 给窗口内所有行赋值 window_touch_type
    window['window_touch_type'] = window_touch_type
    
    # 添加窗口数据到列表
    windowed_data.append(window)

# 合并所有窗口数据
windowed_df = pd.concat(windowed_data, ignore_index=True)

# 保存到新的CSV文件
windowed_df.to_csv(output_file, index=False)

print(f"{output_file} 文件已生成，总行数 = {len(windowed_df)}")


../DATA/labeled_window_dataset.csv 文件已生成，总行数 = 391916


In [17]:
dt = pd.read_csv('../DATA/labeled_window_dataset.csv')
dt[dt['window_touch_type']=='ST']['window_id'].unique()
# grouped = dt.groupby(['window_id', 'window_touch_type', 'touch_type']).size().reset_index(name='count')
# # 展示结果
# print(grouped)
dt[dt['window_id']==4695]

,index,time,tau_J0,tau_J1,tau_J2,tau_J3,tau_J4,tau_J5,tau_J6,tau_J_d0,...,etau_J2,etau_J3,etau_J4,etau_J5,etau_J6,label,block_id,touch_type,window_id,window_touch_type
131460,12027.0,120.260057,-0.188623,-17.220572,-2.077830,17.847666,0.383018,2.499279,-0.120983,0.0,...,2.077830,-17.847666,-0.383018,-2.499279,0.120983,0.0,-1.0,NC,4695,ST
131461,12028.0,120.269998,-0.188623,-17.220572,-2.077830,17.847666,0.383018,2.499279,-0.120983,0.0,...,2.077830,-17.847666,-0.383018,-2.499279,0.120983,0.0,-1.0,NC,4695,ST
131462,12029.0,120.280031,-0.148950,-17.220572,-2.077830,17.847666,0.373863,2.510265,-0.059643,0.0,...,2.077830,-17.847666,-0.373863,-2.510265,0.059643,0.0,-1.0,NC,4695,ST
131463,12030.0,120.289992,-0.188623,-17.145193,-2.077830,17.847666,0.364707,2.499279,-0.120983,0.0,...,2.077830,-17.847666,-0.364707,-2.499279,0.120983,0.0,-1.0,NC,4695,ST
131464,12031.0,120.300025,-0.224328,-17.220572,-2.006419,17.811960,0.364707,2.520336,-0.049572,0.0,...,2.006419,-17.811960,-0.364707,-2.520336,0.049572,0.0,-1.0,NC,4695,ST
131465,12032.0,120.310000,-0.148950,-17.220572,-2.077830,17.811960,0.373863,2.510265,-0.109997,0.0,...,2.077830,-17.811960,-0.373863,-2.510265,0.109997,0.0,-1.0,NC,4695,ST
131466,12033.0,120.320036,-0.148950,-17.184866,-2.077830,17.811960,0.383018,2.510265,-0.028515,0.0,...,2.077830,-17.811960,-0.383018,-2.510265,0.028515,0.0,-1.0,NC,4695,ST
131467,12034.0,120.329992,-0.148950,-17.184866,-2.042125,17.847666,0.383018,2.499279,-0.109997,0.0,...,2.042125,-17.847666,-0.383018,-2.499279,0.109997,0.0,-1.0,NC,4695,ST
131468,12035.0,120.340029,-0.148950,-17.260244,-2.006419,17.847666,0.392173,2.510265,-0.028515,0.0,...,2.006419,-17.847666,-0.392173,-2.510265,0.028515,0.0,-1.0,NC,4695,ST
131469,12036.0,120.349995,-0.224328,-17.299917,-2.077830,17.847666,0.364707,2.489208,-0.131054,0.0,...,2.077830,-17.847666,-0.364707,-2.489208,0.131054,0.0,-1.0,NC,4695,ST
